## Przykłady wzorca Abstract Factory w kodzie Pythona

Wzorzec Abstract Factory tworzy **rodziny powiązanych obiektów** bez określania ich konkretnych klas.

**Kluczowa cecha**: Produkty z jednej fabryki muszą ze sobą współpracować - nie można mieszać produktów z różnych fabryk.

### 1. Django database backends - różne silniki baz danych

Najbardziej znany przykład Abstract Factory w Pythonie.

**Problem**: Aplikacja musi działać z różnymi bazami danych (PostgreSQL, MySQL, SQLite), a każda baza wymaga **rodziny spójnych obiektów** (connection, operations, features).

#### Struktura Django backends

Każdy backend dostarcza całą rodzinę produktów:

```
django.db.backends.postgresql/
    DatabaseWrapper      # konkretny produkt 1
    DatabaseOperations   # konkretny produkt 2  
    DatabaseFeatures     # konkretny produkt 3
    DatabaseClient       # konkretny produkt 4
    DatabaseCreation     # konkretny produkt 5

django.db.backends.mysql/
    DatabaseWrapper      # inna implementacja
    DatabaseOperations   # inna implementacja
    DatabaseFeatures     # inna implementacja
    ...
```

**Te produkty MUSZĄ ze sobą współpracować!**

In [ ]:
# settings
# Demonstracja: różne backend-y dostępne w Django

backends = [
    'django.db.backends.sqlite3',  # to są nazwy modułów
    'django.db.backends.postgresql', 
    'django.db.backends.mysql',
    'django.db.backends.oracle'
]


#### Jak to działa w praktyce

In [ ]:
# Symulacja: wybór fabryki przez konfigurację

# settings.py - wybieramy CAŁĄ RODZINĘ
DATABASE_CONFIG = {
    'ENGINE': 'django.db.backends.postgresql',  # <-- wybór fabryki
    'NAME': 'mydb',
}


#### Dlaczego NIE MOŻNA mieszać produktów?

In [ ]:
# To NIE ZADZIAŁA:
# connection = postgresql.DatabaseWrapper(...)
# operations = mysql.DatabaseOperations(...)  # <-- z innej fabryki!


#### Wymienialność całej rodziny

In [ ]:
# Zmiana ENGINE wymienia WSZYSTKIE produkty

configs = [
    {'ENGINE': 'django.db.backends.postgresql'},
    {'ENGINE': 'django.db.backends.mysql'},
    {'ENGINE': 'django.db.backends.sqlite3'},
]


#### Klient nie zna konkretnych klas

Django ORM (klient) używa abstrakcyjnego interfejsu

```python
# client Django ORM
from django.db import connection

cursor = connection.cursor()           # nie wie czy to PostgreSQL czy MySQL
cursor.execute("SELECT * FROM users")  # abstrakcyjna operacja
result = cursor.fetchall()
```

- Ten kod działa z KAŻDYM backendem
- Zmiana ENGINE w settings.py → zero zmian w kodzie!
- Ten sam kod działa z PostgreSQL, MySQL, SQLite
- Klient nie zna konkretnych klas
- Zmiana fabryki = zmiana w settings.py

### Mapowanie na wzorzec Abstract Factory

**Struktura wzorca:**
- **AbstractFactory**: `django.db.backends` (moduł)
- **ConcreteFactory1**: `django.db.backends.postgresql`
- **ConcreteFactory2**: `django.db.backends.mysql`
- **ConcreteFactory3**: `django.db.backends.sqlite3`

**Produkty:**
- **AbstractProductA**: interfejs `DatabaseWrapper`
- **AbstractProductB**: interfejs `DatabaseOperations`
- **AbstractProductC**: interfejs `DatabaseFeatures`

**Konkretne produkty:**
- **ConcreteProductA1**: `postgresql.DatabaseWrapper`
- **ConcreteProductA2**: `mysql.DatabaseWrapper`
- **ConcreteProductB1**: `postgresql.DatabaseOperations`
- **ConcreteProductB2**: `mysql.DatabaseOperations`

**Klient:**
- Django ORM - używa przez abstrakcyjny interfejs `connection`

### 3. Test: Czy to Abstract Factory?

**Pytania testowe:**

1. **Czy tworzy RODZINĘ powiązanych obiektów?**
   - Wrapper + Operations + Features - TAK ✅

2. **Czy produkty z różnych fabryk można mieszać?**
   - PostgreSQL Wrapper + MySQL Operations = konflikt - NIE ✅

3. **Czy klient zna konkretne klasy?**
   - używa przez `connection` - NIE  ✅
   - 

### Podsumowanie

**Abstract Factory vs inne wzorce:**

| Wzorzec | Tworzy | Spójność rodziny |
|---------|--------|------------------|
| **Factory Method** | Jeden produkt | Nie dotyczy |
| **Abstract Factory** | Rodzina produktów | **Wymagana** |
| **Builder** | Złożony obiekt krok po kroku | Nie dotyczy |

**Kiedy używać Abstract Factory?**
- System musi być niezależny od sposobu tworzenia produktów
- Produkty muszą ze sobą współpracować (rodzina)
- Chcesz wymienić całą rodzinę produktów
- Chcesz ukryć konkretne klasy przed klientem
